In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# kinematics and dynamics code of these casadi objects can be found in https://github.com/edxmorgan/diff_uv project
# these can be replaced with your preferred uv dynamics
fb_id_func = cs.Function.load("float_base.casadi")
J_uv_func = cs.Function.load("J_uv.casadi")

# ned jacobian for uv kinematics
J_uv = J_uv_func(base_ss.eul)

z_SX = base_ss.p_n[2]

# base bias force expressions
zero_acc = [0,0,0,0,0,0]
b_F = fb_id_func(z_SX, base_ss.eul, base_ss.v_base, zero_acc, base_ss.sim_p)

# base inverse dynamics expressions
tau_F = fb_id_func(z_SX, base_ss.eul, base_ss.v_base, base_ss.a_base, base_ss.sim_p)

# uvms bias force and Inertia
B = rig_dyn.get_bias_force(gravity=9.81, floating_base_bias_f=b_F, coupled=True)
H = rig_dyn.get_inertia_matrix(gravity=9.81, floating_base_id=tau_F, floating_base_bias_f=b_F, coupled=True)

floating_base found
floating_base found
floating_base found


In [3]:
H_eval = cs.Function(
    'H', [z_SX, base_ss.eul, arm_ss.q, arm_ss.q_dot, base_ss.v_base, base_ss.uv_u, base_ss.base_T, arm_ss.sim_p, base_ss.sim_p], [H])
B_eval = cs.Function(
    'B', [z_SX, base_ss.eul, arm_ss.q, arm_ss.q_dot, base_ss.v_base, base_ss.uv_u, base_ss.base_T, arm_ss.sim_p, base_ss.sim_p], [B])

In [4]:
qq, qqd = [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]
v0 = [0, 0, 0, 0, 0, 0]
eul0 = [0, 0, 0]
uv_u = [0, 0, 0, 0, 0, 0]

H_eval(blue.below_surface, eul0, qq, qqd, v0, uv_u, alpha.base_T0, alpha.sim_p, blue.sim_p)

DM(
[[0.28, 0, 0, 0, -0.23, 0, 0.00428134, 0.0189567, -0.00611939, 0.000187603], 
 [0, 0.28, 0, 0.23, 0, 0, -0.00777851, -0.0183087, 0.0176397, -0.000102593], 
 [0, 0, 0.28, 0, 0, 0, -0.0286729, -0.0112978, 0.00339453, 0.00334353], 
 [0, 0.23, 0, 17, 0, 0, -0.0253369, 0.0182215, -0.0110527, -0.000490842], 
 [-0.23, 0, 0, 0, 24.2, 0, -0.0275437, -0.0606496, 0.0187418, -0.000870158], 
 [0, 0, 0, 0, 0, 26.07, -1.12198e-05, 0.00585898, 0.00776247, -3.54456e-07], 
 [0, 0, 0, 0, 0, 0, 0.0225364, 1.24241e-06, -3.38627e-07, -1.53776e-06], 
 [0, 0, 0, 0, 0, 0, 2.32282e-06, 0.022534, -8.73647e-06, 2.2922e-08], 
 [0, 0, 0, 0, 0, 0, -6.74167e-07, 1.13485e-06, 0.022535, 8.72574e-09], 
 [0, 0, 0, 0, 0, 0, -2.99173e-08, 1.77816e-07, 3.90861e-08, 0.00340423]])

In [5]:
# for H's decoupled version, compare to these matrix
#[Hv 0]
#[0 Hm]

# vehicle base inertia matrix, Hv :
# [[ 0.28  0.    0.    0.   -0.23  0.  ]
#  [ 0.    0.28  0.    0.23  0.    0.  ]
#  [ 0.    0.    0.28  0.    0.    0.  ]
#  [ 0.    0.23  0.   17.    0.    0.  ]
#  [-0.23  0.    0.    0.   24.2   0.  ]
#  [ 0.    0.    0.    0.    0.   26.07]]

# Manipulator inertia matrix, Hm :
# M = rig_dyn.get_inertia_matrix(gravity=9.81, floating_base_id=None, floating_base_bias_f=None, coupled=False)
# M_eval = cs.Function('H', [ss.q, ss.q_dot,  ss.base_T, ss.sim_p], [M])
# M_eval(qq, qqd, alpha.base_T0, alpha.sim_p)

# DM(
# [[0.0225364, 1.24241e-06, -3.38627e-07, -1.53776e-06], 
#  [2.32282e-06, 0.022534, -8.73647e-06, 2.2922e-08], 
#  [-6.74167e-07, 1.13485e-06, 0.022535, 8.72574e-09], 
#  [-2.99173e-08, 1.77816e-07, 3.90861e-08, 0.00340423]])

In [6]:
# for B_F's decoupled version, compare to these matrix
#[B_Fv]
#[C_Fm]

# B_F_eval = cs.Function('B_F_v', [z_SX, ss.eul, ss.v_base], [b_F]) # for numerical & symbolic use
# B_Fv = B_F_eval(blue.below_surface, eul0, v0)
# DM([0, 0, 0, 0, 0, 1.985])

# C = rig_dyn.get_bias_force(gravity=9.81, floating_base_bias_f=None, coupled=False)
# C_eval = cs.Function('C', [ss.q, ss.q_dot,  ss.base_T, ss.sim_p], [C])
# C_eval(qq, qqd, alpha.base_T0, alpha.sim_p)